# Arctic-calibrated Brunt equation

## Jason Kniss
## Feb 14 2024

### Calibrate the Brunt equation using data from the Eureka site

Import libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta